<a href="https://colab.research.google.com/github/Kzis/my-saprk/blob/main/my_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Initial Spark**

In [31]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [32]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [33]:
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [34]:
!pip install -q findspark

In [35]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [36]:
import findspark
findspark.init()

In [37]:
findspark.find()

'/content/spark-3.0.0-bin-hadoop3.2'

In [38]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [39]:
spark

# **Spark DataFrame Basics**

In [40]:
df = spark.read.json('people.json')

In [41]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

